### Open Flights Data Wrangling

To practice, you are going to wrangle data from OpenFlights.  You can read about it here: 

http://openflights.org/data.html

This includes three main files, one for each airport, one for each airline, and one for each route.  They can be merged or joined with the appropriate fields.  I have modified the files slightly to include a header row in the .dat files, which makes it a bit easier for you.  

You are required to work through the problems below.  This may take some time.  Be persistent, and ask questions or seek help as needed.  

In [54]:
import pandas as pd
import numpy as np

In [55]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("airports.dat", header=None, na_values='\\N')
airports.columns = ["id", "name", "city", "country", "iata", "icao", "latitude", "longitude", "altitude","timezone", "dst", "tz", "type", "source"]

# Read in the airlines data.
airlines = pd.read_csv("airlines.dat", header=None, na_values='\\N')
airlines.columns = ["id", "name", "alias", "iata", "icao", "callsign", "country", "active"]

# Read in the routes data.
routes = pd.read_csv("routes.dat", header=None, na_values='\\N')
routes.columns = ["airline", "airline_id", "source", "source_id", "dest", "dest_id", "codeshare", "stops", "equipment"]

1) Start by seeing what's in the data.  What columns are there?  What data types are the columns?  

Remember, 'object' means it is a string, while the numerical values can be floats or ints.  Sometimes you will have problems if it reads numeric data in as strings.  If that happens, you can use the function .astype() to convert it.  Look it up in the pandas API to get more details

In [13]:
airports.dtypes

id             int64
name          object
city          object
country       object
iata          object
icao          object
latitude     float64
longitude    float64
altitude       int64
timezone     float64
dst           object
tz            object
type          object
source        object
dtype: object

In [72]:
airlines.dtypes

id           int64
name        object
alias       object
iata        object
icao        object
callsign    object
country     object
active      object
dtype: object

In [53]:
routes.dtypes

airline        object
airline_id    float64
source         object
source_id     float64
dest           object
dest_id       float64
codeshare      object
stops           int64
equipment      object
dtype: object

2) Select just the routes that go to or from Lexington Bluegrass Airport, and store them in their own dataframe.  

The airport code is LEX.  You should have a much smaller dataframe.  How many inbound routes and how many outbound routes are there? 

In [56]:

# Select the LEX routes, then join the source airports
lex_routeso = routes[routes['source']=="LEX"]
lex_routeso = pd.merge(lex_routeso, airports, left_on='source_id', right_on='id', how='left')

# join the destination airports.  Here we need to use the suffixes option, because 
# the column names overlap, and we want to distinguish between source and dest
lex_routeso = pd.merge(lex_routeso, airports, 
                      left_on='dest_id', 
                      right_on='id', 
                      how='left', 
                      suffixes=['_source','_dest'])
# It looks like source has some duplicate names.  Drop the values from the airports
# file ane keep the one from the routes file
lex_routeso = lex_routeso.drop(['source_y','source'], axis=1)
lex_routeso = lex_routeso.rename(columns={'source_x': 'source'})
# here is what our data looks like
lex_routeso



,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,id_source,...,country_dest,iata_dest,icao_dest,latitude_dest,longitude_dest,altitude_dest,timezone_dest,dst_dest,tz_dest,type_dest
0,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,United States,ATL,KATL,33.636700,-84.428101,1026,-5.0,A,America/New_York,airport
1,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ,4017,...,United States,CLT,KCLT,35.214001,-80.943100,748,-5.0,A,America/New_York,airport
2,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4,4017,...,United States,DFW,KDFW,32.896801,-97.038002,607,-6.0,A,America/Chicago,airport
3,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4,4017,...,United States,ORD,KORD,41.978600,-87.904800,672,-6.0,A,America/Chicago,airport
4,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9,4017,...,United States,ATL,KATL,33.636700,-84.428101,1026,-5.0,A,America/New_York,airport
5,DL,2009.0,LEX,4017.0,ATL,3682.0,NaN,0,M88 717,4017,...,United States,ATL,KATL,33.636700,-84.428101,1026,-5.0,A,America/New_York,airport
6,DL,2009.0,LEX,4017.0,DCA,3520.0,Y,0,CRJ,4017,...,United States,DCA,KDCA,38.852100,-77.037697,15,-5.0,A,America/New_York,airport
7,DL,2009.0,LEX,4017.0,DTW,3645.0,Y,0,CR7 CRJ CR9,4017,...,United States,DTW,KDTW,42.212399,-83.353401,645,-5.0,A,America/New_York,airport
8,DL,2009.0,LEX,4017.0,LGA,3697.0,NaN,0,ERJ,4017,...,United States,LGA,KLGA,40.777199,-73.872597,21,-5.0,A,America/New_York,airport
9,DL,2009.0,LEX,4017.0,MSP,3858.0,Y,0,CRJ,4017,...,United States,MSP,KMSP,44.882000,-93.221802,841,-6.0,A,America/Chicago,airport


In [57]:
# Select the LEX routes, then join the source airports
lex_routesi = routes[(routes['dest']=="LEX")]
lex_routesi = pd.merge(lex_routesi, airports, left_on='dest_id', right_on='id', how='left')

# join the destination airports.  Here we need to use the suffixes option, because 
# the column names overlap, and we want to distinguish between source and dest
lex_routesi = pd.merge(lex_routesi, airports, 
                      left_on='source_id', 
                      right_on='id', 
                      how='left', 
                      suffixes=['_dest','_source'])

# It looks like source has some duplicate names.  Drop the values from the airports
# file ane keep the one from the routes file
lex_routesi = lex_routesi.drop(['source_y','source'], axis=1)
lex_routesi = lex_routesi.rename(columns={'source_x': 'source'})
# here is what our data looks like
lex_routesi


,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment,id_dest,...,country_source,iata_source,icao_source,latitude_source,longitude_source,altitude_source,timezone_source,dst_source,tz_source,type_source
0,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ,4017,...,United States,ATL,KATL,33.636700,-84.428101,1026,-5.0,A,America/New_York,airport
1,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7,4017,...,United States,CLT,KCLT,35.214001,-80.943100,748,-5.0,A,America/New_York,airport
2,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,4017,...,United States,DFW,KDFW,32.896801,-97.038002,607,-6.0,A,America/Chicago,airport
3,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,4017,...,United States,ORD,KORD,41.978600,-87.904800,672,-6.0,A,America/Chicago,airport
4,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717,4017,...,United States,ATL,KATL,33.636700,-84.428101,1026,-5.0,A,America/New_York,airport
5,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717,4017,...,United States,ATL,KATL,33.636700,-84.428101,1026,-5.0,A,America/New_York,airport
6,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ,4017,...,United States,DCA,KDCA,38.852100,-77.037697,15,-5.0,A,America/New_York,airport
7,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717,4017,...,United States,DTW,KDTW,42.212399,-83.353401,645,-5.0,A,America/New_York,airport
8,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ,4017,...,United States,LGA,KLGA,40.777199,-73.872597,21,-5.0,A,America/New_York,airport
9,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ,4017,...,United States,MSP,KMSP,44.882000,-93.221802,841,-6.0,A,America/Chicago,airport


In [93]:
lex_routeso['source'].value_counts()

LEX    20
Name: source, dtype: int64

In [94]:
lex_routesi['dest'].value_counts()

LEX    20
Name: dest, dtype: int64

3) Now let's look at which airlines operate in and out of Lexington.  To do this, you need to merge the airline dataframe to the route dataframe.  

How many routes does each airline have?  The value_counts() method may be useful for answering this question.  

In [90]:
rlines = pd.merge(airlines, lex_routes, left_on='id', right_on='airline_id')
rlines

,id,name,alias,iata,icao,callsign,country,active,airline,airline_id,...,country_source,iata_source,icao_source,latitude_source,longitude_source,altitude_source,timezone_source,dst_source,tz_source,type_source
0,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,United States,CLT,KCLT,35.214001,-80.943100,748,-5.0,A,America/New_York,airport
1,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,United States,DFW,KDFW,32.896801,-97.038002,607,-6.0,A,America/Chicago,airport
2,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,United States,ORD,KORD,41.978600,-87.904800,672,-6.0,A,America/Chicago,airport
3,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,United States,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport
4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,United States,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport
5,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,United States,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport
6,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,FLL,KFLL,26.072599,-80.152702,9,-5.0,A,America/New_York,airport
7,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,PGD,KPGD,26.920200,-81.990501,26,-5.0,A,America/New_York,airport
8,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,PIE,KPIE,27.910200,-82.687401,11,-5.0,A,America/New_York,airport
9,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,SFB,KSFB,28.777599,-81.237503,55,-5.0,A,America/New_York,airport


In [48]:
rlines['name'].value_counts()

Delta Air Lines             10
Allegiant Air                8
US Airways                   6
American Airlines            6
United Airlines              4
KLM Royal Dutch Airlines     2
Air France                   2
Pinnacle Airlines            2
Name: name, dtype: int64

4) It looks like there are some international airlines with Lexington routes.  To look at how many routes they have, create a new column in your dataframe called 'International', which is set to Y for an overseas airline and N for a domestic airline.  Calculate the percent of routes with an overseas airline.  

In [69]:
def clean_ar(value):
    if isinstance(value, str):
        end = value.find('United States')
        if end == -1:
            return 'Y'
        else:
            return 'N'
rlines['international'] = rlines['country'].map(clean_ar)

rlines

,id,name,alias,iata,icao,callsign,country,active,airline,airline_id,...,iata_source,icao_source,latitude_source,longitude_source,altitude_source,timezone_source,dst_source,tz_source,type_source,international
0,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,CLT,KCLT,35.214001,-80.943100,748,-5.0,A,America/New_York,airport,N
1,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,DFW,KDFW,32.896801,-97.038002,607,-6.0,A,America/Chicago,airport,N
2,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,ORD,KORD,41.978600,-87.904800,672,-6.0,A,America/Chicago,airport,N
3,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport,N
4,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport,N
5,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport,N
6,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,FLL,KFLL,26.072599,-80.152702,9,-5.0,A,America/New_York,airport,N
7,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,PGD,KPGD,26.920200,-81.990501,26,-5.0,A,America/New_York,airport,N
8,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,PIE,KPIE,27.910200,-82.687401,11,-5.0,A,America/New_York,airport,N
9,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,SFB,KSFB,28.777599,-81.237503,55,-5.0,A,America/New_York,airport,N


In [50]:
prcnt = ((rlines['international'].str.count('Y').sum()) / (rlines['international'].value_counts().sum())) * 100
prcnt

10.0

5) Actually, it looks like a bunch of these routes are codeshares.  That means they are marketed by this airline, but operated by a different airline.  See the note in the data documentation on openflights.org/data.  The implication of this is that there are duplicates.

Can you figure out which ones are duplicates?  Can you then create a dataframe with only the unique routes?  How many unique inbound and outbound routes are there? 

Remember, someone has to operate the flight, so if all the routes to/from a particular airport are listed as codeshares, then something is funny...

It is also possible that more than one airline actually operates a route between the same two airports. (Having this sort of competition generally means that you will get better fares as a traveler.)  It may not be obvious what is actually in the data set, so dig or do external research as needed.  

In [92]:
uniques=rlines[(~rlines.duplicated(subset=['source','dest'], keep=False)) | (rlines['codeshare'].isnull())]
uniques

,id,name,alias,iata,icao,callsign,country,active,airline,airline_id,...,country_source,iata_source,icao_source,latitude_source,longitude_source,altitude_source,timezone_source,dst_source,tz_source,type_source
0,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y,AA,24.0,...,United States,CLT,KCLT,35.214001,-80.943100,748,-5.0,A,America/New_York,airport
6,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,FLL,KFLL,26.072599,-80.152702,9,-5.0,A,America/New_York,airport
7,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,PGD,KPGD,26.920200,-81.990501,26,-5.0,A,America/New_York,airport
8,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,PIE,KPIE,27.910200,-82.687401,11,-5.0,A,America/New_York,airport
9,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,SFB,KSFB,28.777599,-81.237503,55,-5.0,A,America/New_York,airport
10,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport
11,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport
12,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport
13,35,Allegiant Air,NaN,G4,AAY,ALLEGIANT,United States,Y,G4,35.0,...,United States,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport
16,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y,DL,2009.0,...,United States,ATL,KATL,33.636700,-84.428101,1026,-5.0,A,America/New_York,airport
